# 使用 LLM API

本章节主要介绍四种大语言模型（ChatGPTAPI、文心一言、讯飞星火、智谱 GLM）的 API 申请指引和 Python 版本的原生 API 调用方法。如果你需要在 LangChain 中使用 LLM，可以参照第四章第一节的调用方式。

## 1. 使用 ChatGPT

ChatGPT，发布于 2022 年 11 月，是目前火热出圈的大语言模型（Large Language Model，LLM）的代表产品。在 2022 年底，也正是 ChatGPT 的惊人表现引发了 LLM 的热潮。时至目前，由 OpenAI 发布的 GPT-4 仍然是 LLM 性能上限的代表，ChatGPT 也仍然是目前使用人数最多、使用热度最大、最具发展潜力的 LLM 产品。事实上，在圈外人看来，ChatGPT 即是 LLM 的代称。

OpenAI 除发布了免费的 Web 端产品外，也提供了多种 ChatGPT API，支持开发者通过 Python 或 Request 请求来调用 ChatGPT，向自己的服务中嵌入 LLM 的强大能力。可选择的主要模型包括 ChatGPT-3.5 和 GPT-4，但每个模型也存在多个上下文版本，例如 ChatGPT-3.5 就有最原始的 4K 上下文长度的模型，也有 16K 上下文长度的模型 gpt-turbo-16k-0613。

### 1.1 API 申请指引

#### 获取并配置 OpenAI API key

OpenAI API 调用服务是付费的，每一个开发者都需要首先获取并配置 OpenAI API key，才能在自己构建的应用中访问 ChatGPT。我们将在这部分简述如何获取并配置 OpenAI API key。

在获取 OpenAI API key 之前我们需要[OpenAI 官网](https://openai.com/)中注册一个账号。这里假设我们已经有了 OpenAI 账号，先在[OpenAI 官网](https://openai.com/)登录，登录后如下图所示：

<p align="center">
  <img src="../../figures/C2-2-openai-choose.png" width="1000" alt="OpenAI 官网登录后选择 API">
</p>

我们选择 `API`，然后点击左侧边栏的`API keys`，如下图所示：

<p align="center">
  <img src="../../figures/C2-2-openai-get-key.png" width="1000" alt="OpenAI 获取 API key">
</p>

点击 `Create new secret key` 按钮创建 OpenAI API key ，我们将创建好的 OpenAI API key 复制以此形式 `OPENAI_API_KEY="sk-..."` 保存到 `.env` 文件中，并将 `.env` 文件保存在项目根目录下。

下面是读取 `.env` 文件的代码：

In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。

# find_dotenv() 寻找并定位 .env 文件的路径
# load_dotenv() 读取该 .env 文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 如果你需要通过代理端口访问，还需要做如下配置
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
os.environ["HTTP_PROXY"] = 'http://127.0.0.1:7890'

# 获取环境变量 OPENAI_API_KEY
openai.api_key = os.environ['OPENAI_API_KEY']

### 1.2 调用 OpenAI API

调用 ChatGPT 需要使用 [ChatCompletion API](https://platform.openai.com/docs/api-reference/chat)，该 API 提供了 ChatGPT 系列模型的调用，包括 ChatGPT-3.5，GPT-4 等。

ChatCompletion API 调用方法如下：

In [ ]:
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    # 获取环境变量 OPENAI_API_KEY
    api_key=os.environ.get("OPENAI_API_KEY"),
)

# 导入所需库
# 注意，此处我们假设你已根据上文配置了 OpenAI API Key，如没有将访问失败
completion = client.chat.completions.create(
    # 创建一个 ChatCompletion
    # 调用模型：ChatGPT-3.5
    model="gpt-3.5-turbo",
    # message 是你的 prompt
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello!"}
    ]
)

调用该 API 会返回一个 ChatCompletion 对象，其中包括了回答文本、创建时间、id 等属性。我们一般需要的是回答文本，也就是回答对象中的 content 信息。

In [ ]:
completion

In [ ]:
print(completion.choices[0].message.content)

此处我们详细介绍调用 API 常会用到的几个参数：

    · model，即调用的模型，一般取值包括“gpt-3.5-turbo”（ChatGPT-3.5）、“gpt-3.5-turbo-16k-0613”（ChatGPT-3.5 16K 版本）、“gpt-4”（ChatGPT-4）。注意，不同模型的成本是不一样的。

    · messages，即我们的 prompt。ChatCompletion 的 messages 需要传入一个列表，列表中包括多个不同角色的 prompt。我们可以选择的角色一般包括 system：即前文中提到的 system prompt；user：用户输入的 prompt；assistant：助手，一般是模型历史回复，作为提供给模型的参考内容。

    · temperature，温度。即前文中提到的 Temperature 系数。

    · max_tokens，最大 token 数，即模型输出的最大 token 数。OpenAI 计算 token 数是合并计算 Prompt 和 Completion 的总 token 数，要求总 token 数不能超过模型上限（如默认模型 token 上限为 4096）。因此，如果输入的 prompt 较长，需要设置较小的 max_token 值，否则会报错超出限制长度。

OpenAI 提供了充分的自定义空间，支持我们通过自定义 prompt 来提升模型回答效果，如下是一个简单的封装 OpenAI 接口的函数，支持我们直接传入 prompt 并获得模型的输出：

In [ ]:
# 一个封装 OpenAI 接口的函数，参数为 prompt，返回对应结果
def get_completion(prompt, model="gpt-3.5-turbo", temperature = 0):
    '''
    prompt: 对应的提示词
    model: 调用的模型，默认为 gpt-3.5-turbo，也可以按需选择 gpt-4 等其他模型
    temperature: 模型输出的温度系数，控制输出的随机程度，取值范围是 0~2。温度系数越低，输出内容越一致。
    '''
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    # 调用 OpenAI 的 ChatCompletion 接口
    return response.choices[0].message.content


在上述函数中，我们封装了 messages 的细节，仅使用 user prompt 来实现调用。在简单场景中，该函数足够满足使用需求。

## 2. 使用文心一言

`文心一言`，由百度于 2023 年 3 月 27 日推出的中文大模型，是目前国内大语言模型的代表产品。受限于中文语料质量差异及国内计算资源、计算技术瓶颈，文心一言在整体性能上距离 ChatGPT 仍有一定差异，但在中文语境下已展现出了较为优越的性能。文心一言所考虑的落地场景包括多模态生成、文学创作等多种商业场景，其目标是在中文语境下赶超 ChatGPT。当然，要真正战胜 ChatGPT，百度还有很长的路要走；但在生成式 AI 监管较为严格的国内，作为第一批被允许向公众开放的生成式 AI 应用，文心一言相对无法被公开使用的 ChatGPT 还是具备一定商业上的优势。

百度同样提供了文心一言的 API 接口，其在推出大模型的同时，也推出了 `文心千帆` 企业级大语言模型服务平台，包括了百度整套大语言模型开发工作链。对于不具备大模型实际落地能力的中小企业或传统企业，考虑文心千帆是一个可行的选择。当然，本教程仅包括通过文心千帆平台调用文心一言 API，对于其他企业级服务不予讨论。

### 2.1 API 申请指引

#### 获取密钥

要调用文心一言 API，需要先获取文心一言调用密钥，在代码中需要配置自己的密钥才能实现对模型的调用，具体参考 [API调用流程](https://cloud.baidu.com/doc/WENXINWORKSHOP/s/yloieb01t)。

百度文心有两层密钥认证，第一层是从百度智能云账户中获取的 API_Key 和 Secret_Key，每一个账户可以创建若干个应用，每个应用会对应一个 API_Key 和 Secret_Key。

在获取完成 API_Key 和 Secret_Key 后，还需要基于这两个 Key 去获取 access_token 值。access_token 是第二层认证，基于 access_token 即可调用百度文心大模型。**注意：access_token 默认有效期 30 天，生产环境调用需要及时刷新。**通过上述两层密钥认证，可以进一步提高企业服务的安全性。

当然，在本教程中，我们并不需要将两层密钥分离。我们将在该部分简述如何获取 API_Key、Secret_Key 以及如何基于 Key 获取 access_token 值，后续我们将直接通过 access_token 来调用文心大模型。

![](../../figures/C2-2-baidu_qianfan_1.png)

首先我们需要进入[文心千帆服务平台](https://console.bce.baidu.com/qianfan/overview)，点击上述应用接入按钮，创建一个调用文心大模型的应用。注意，你需要首先有一个经过实名认证的百度账号。

![](../../figures/C2-2-baidu_qianfan_2.png)

接着点击“去创建”按钮，进入应用创建界面：

![](../../figures/C2-2-baidu_qianfan_3.png)

简单输入基本信息，选择默认配置，创建应用即可。

![](../../figures/C2-2-baidu_qianfan_4.png)

创建完成后，我们可以在控制台看到创建的应用的 `AppID`、`API Key`、`Secret Key`。使用这里的 `API Key` 和 `Secret Key` 即可进行 access_token 的获取。

access_token 需要通过代码 post 访问指定网址得到：

In [ ]:
import requests
import json

def get_access_token():
    """
    使用 API Key，Secret Key 获取 access_token，替换下列示例中的应用 api_key、应用 secret_key
    """
    # 指定网址
    url = "https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id={api_key}&client_secret={secret_key}"
    # 设置 POST 访问
    payload = json.dumps("")
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    # 通过 POST 访问获取账户对应的 access_token
    response = requests.request("POST", url, headers=headers, data=payload)
    return response.json().get("access_token")

In [ ]:
get_access_token()

通过上述代码，即可获取到账户对应的 access_token，后续使用 access_token 即可调用百度文心大模型。

### 2.2 调用文心千帆 API

在完成 access_token 的获取后，可以同样通过 POST 访问来调用百度文心原生接口：

In [ ]:
def get_wenxin(prompt):
    # 调用接口（替换请求 url 中的 access_token 变量）
    url = "https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/chat/completions?access_token={access_token}"
    # 配置 POST 参数
    payload = json.dumps({
        "messages": [
            {
                "role": "user",# user prompt
                "content": "{}".format(prompt)# 输入的 prompt
            }
        ]
    })
    headers = {
        'Content-Type': 'application/json'
    }
    # 发起请求
    response = requests.request("POST", url, headers=headers, data=payload)
    # 返回的是一个 Json 字符串
    resp = json.loads(response.text)
    print(resp)
    print(resp["result"])


百度文心同样支持在传入参数的 messages 字段中配置 user、assistant 两个成员角色的 prompt，但与 OpenAI 的 prompt 格式不同的是，模型人设是通过另一个参数 system 字段传入的，而不是在 messages 字段中。

同时，百度文心返回的数据也是一个 JSON 字符串，我们可以解析其中的 result 属性来获取返回数据。

In [ ]:
get_wenxin("你好")

百度千帆提供了多种模型接口供调用，此处我们主要使用 `ERNIE-Bot` 模型的 chat 接口，也就是常说的百度文心大模型。此处简要介绍文心大模型接口的常用参数：

    · messages，即调用的 prompt。文心的 messages 配置与 ChatGPT 有一定区别，其不支持 max_token 参数，由模型自行控制最大 token 数，message中的 content 总长度、functions 和 system 字段总内容不能超过 20480 个字符，且不能超过 5120 tokens，否则模型就会自行对前文依次遗忘。文心的 messages 有以下几点要求：① 一个成员为单轮对话，多个成员为多轮对话；② 最后一个 message 为当前对话，前面的 message 为历史对话；③ 成员数目必须为奇数，message 中的 role 必须依次是 user、assistant。

    · stream，是否使用流式传输。

    · temperature，温度系数，默认 0.8，文心的 temperature 参数要求范围为 (0, 1.0]，不能设置为 0。

我们同样封装一个调用百度文心大模型的函数供之后使用：

In [ ]:
# 一个封装 Wenxin 接口的函数，参数为 Prompt，返回对应结果
def get_completion_weixin(prompt, temperature = 0.1, access_token = ""):
    '''
    prompt: 对应的提示词
    temperature：温度系数
    access_token：已获取到的密钥
    '''
    url = f"https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/chat/completions?access_token={access_token}"
    # 配置 POST 参数
    payload = json.dumps({
        "messages": [
            {
                "role": "user",# user prompt
                "content": "{}".format(prompt)# 输入的 prompt
            }
        ],
        "temperature" : temperature
    })
    headers = {
        'Content-Type': 'application/json'
    }
    # 发起请求
    response = requests.request("POST", url, headers=headers, data=payload)
    # 返回的是一个 JSON 字符串
    resp = json.loads(response.text)
    # print(resp)
    return resp["result"]

In [ ]:
prompt = "你好"
access_token = "xxx"
get_completion_weixin(prompt, access_token=access_token)

## 3. 使用讯飞星火

讯飞星火认知大模型，由科大讯飞于 2023 年 5 月推出的中文大模型，也是国内大模型的代表产品之一。同样，受限于中文语境与算力资源，星火在使用体验上与 ChatGPT 还存在差异，但是，作为与文心不分伯仲的国内中文大模型，仍然值得期待与尝试。相较于存在显著资源、技术优势的百度，科大讯飞想要杀出重围，成为国内大模型的佼佼者，需要充分利用相对优势，至少目前来看，星火并未掉队。

相对于文心与 GPT 几乎完全一致的 API 调用方式，星火 API 需要使用 WebSocket 来进行调用，对企业友好，但对初学者、新手开发者来说调用难度较大。本章节将指导开发者如何将星火 API 封装为可直接通过 request 调用的 API 接口从而和其他大模型保持一致。

### 3.1 API 申请指引

相对于文心复杂的申请流程，讯飞的权限申请显得简单易操作得多：

首先进入到[讯飞星火认知大模型首页](https://xinghuo.xfyun.cn/sparkapi)，点击“免费试用”：

我们可以使用 [Datawhale 提供的专属链接](https://xinghuo.xfyun.cn/sparkapi?ch=dwKeloHY)，通过该链接进入可以获得更多的免费额度，点击“免费试用”：

![](../../figures/C2-2-spark_1.png)

![](../../figures/C2-2-spark_2.png)

如果是没有领取过免费试用包的用户，可以领取到 100000 tokens 的试用量，完成个人身份认证后，还可以免费领取 2000000 tokens 的试用量。完成领取后，点击进入控制台并创建应用，创建完成后，就可以看到我们获取到的 `APPID`、`APISecret` 和 `APIKey` 了：

![](../../figures/C2-2-spark_3.png)

### 3.2 调用讯飞星火 API

星火 API 需要通过 WebSocket 进行连接，相对来说配置较为复杂，讯飞给出了配置示例 SparkApi.py 和连接示例 test.py，此处我们仅讲解 test.py 里的调用逻辑，配置示例代码直接使用即可。

注意，需要在环境中安装配置示例中的所需第三方库。

首先我们需要配置秘钥信息：

In [ ]:
import SparkApi

#以下密钥信息从控制台获取
appid = ""     #填写控制台中获取的 APPID 信息
api_secret = ""   #填写控制台中获取的 APISecret 信息
api_key = ""    #填写控制台中获取的 APIKey 信息

#用于配置大模型版本
#domain = "general"   # v1.5版本
domain = "generalv3.5"    # v3.5版本

#云端环境的服务地址
#Spark_url = "ws://spark-api.xf-yun.com/v1.1/chat"  # v1.5环境的地址
Spark_url = "ws://spark-api.xf-yun.com/v3.5/chat"  # v3.5环境的地址

星火的调用传参和文心类似，也需要传入一个列表，列表中包括 role 和 content，我们首先定义一个从用户输入 prompt 生成传入参数的函数：

In [ ]:
def getText(role, content, text = []):
    # role 是指定角色，content 是 prompt 内容
    jsoncon = {}
    jsoncon["role"] = role
    jsoncon["content"] = content
    text.append(jsoncon)
    return text

接着，我们将一个用户输入 prompt 封装为这样一个传入参数列表：

In [ ]:
question = getText("user", "你好")
question

然后再调用 SparkApi.py 中封装的 main 函数即可：

In [ ]:
response = SparkApi.main(appid,api_key,api_secret,Spark_url,domain,question)
response

### 3.3 调用讯飞星火 API

由于星火使用了 WebSocket 连接，不能直接使用 request 来进行访问，同其他大模型 API 访问方式具有一定差异。我们可以通过 FastAPI 将星火 API 封装成本地 API，从而实现统一的模型调用方式。我们在《附2 将大模型 API 封装成本地 API》中讲解了如何实现本地 API 的封装，此处仅讲解如何启动并调用本地 API。

在我们完成本地 API 的封装后（spark_api.py），我们可以通过 uvicorn 命令启动：

In [ ]:
! uvicorn spark_api:app

启动之后，默认会在本地 8000 端口开启 API 服务。

启动 API 之后，我们可以向本地 8000 端口发起 Request 请求来访问 API：

In [ ]:
import requests

api_url = "http://127.0.0.1:8000/spark"
headers = {"Content-Type": "application/json"}
data = {
    "prompt" : "你好",
    "temperature" : 0.2,
    "max_tokens" : 3096}

response = requests.post(api_url, headers=headers, json=data)
response.text

同样，我们可以用一个函数来封装 requests 访问的细节：

In [ ]:
def get_completion_spark(prompt, temperature = 0.1, max_tokens = 4096):

    api_url = "http://127.0.0.1:8000/spark"
    headers = {"Content-Type": "application/json"}
    data = {
        "prompt" : prompt,
        "temperature" : temperature,
        "max_tokens" : max_tokens}

    response = requests.post(api_url, headers=headers, json=data)
    return response.text

In [ ]:
get_completion_spark("你好")

## 4. 使用智谱 GLM

智谱 AI 是由清华大学计算机系技术成果转化而来的公司，致力于打造新一代认知智能通用模型。公司合作研发了双语千亿级超大规模预训练模型 GLM-130B，并构建了高精度通用知识图谱，形成数据与知识双轮驱动的认知引擎，基于此模型打造了 ChatGLM（chatglm.cn）。

ChatGLM 系列模型，包括 ChatGLM-130B、ChatGLM-6B 和 ChatGLM2-6B（ChatGLM-6B 的升级版本）模型，支持相对复杂的自然语言指令，并且能够解决困难的推理类问题。其中，ChatGLM-6B 模型来自 Huggingface 上的下载量已经超过 300w（截至 2023 年 6 月 24 日统计数据），该模型在 Hugging Face (HF) 全球大模型下载榜中连续 12 天位居第一名，在国内外的开源社区中产生了较大的影响。

### 4.1 API 申请指引

首先进入到 [智谱AI开放平台](https://open.bigmodel.cn/overview)，点击“开始使用”或者“开发者工具台”进行注册：

![](../../figures/C2-2-zhipuai_home.png)

新注册的用户可以免费领取有效期 1 个月的 100w token 的体验包，进行个人实名认证后，还可以额外领取 400w token 体验包。智谱 AI 提供了 GLM-4 和 GLM-3-Turbo 这两种不同模型的体验入口，可以点击立即体验按钮直接体验。对于需要使用 API key 来搭建应用的话，需要点击右侧的查看 API key按钮，就会进入到我们个人的 API 管理列表中。

![智谱 AI 控制台](../../figures/C2-2-zhipuai_overview.png)


在该界面，我们就可以看到我们获取到的 API 所对应的应用名字和 `API key` 了。

我们可以点击 `添加新的 API key` 并输入对应的名字即可生成新的 API key。

![智谱 AI api 管理](../../figures/C2-2-zhipuai_api.png)

### 4.2 调用智谱 GLM API

智谱 AI 提供了 SDK 和原生 HTTP 来实现模型 API 的调用，建议使用 SDK 进行调用以获得更好的编程体验。

注意，需要在环境中安装配置示例中的所需第三方库。

首先我们需要配置密钥信息：

In [ ]:
!pip install zhipuai

In [ ]:
from zhipuai import ZhipuAI

client = ZhipuAI(api_key="")#填写控制台中获取的 APIKey 信息

model = "glm-4" #用于配置大模型版本

智谱的调用传参和其他类似，也需要传入一个列表，列表中包括 role 和 prompt，我们首先定义一个从用户输入 prompt 生成传入参数的函数：

In [ ]:
def getText(role, content, text = []):
    # role 是指定角色，content 是 prompt 内容
    jsoncon = {}
    jsoncon["role"] = role
    jsoncon["content"] = content
    text.append(jsoncon)
    return text

接着，我们将一个用户输入 prompt 封装为这样一个传入参数列表：

In [ ]:
question = getText("user", "你好")
question

然后再调用 zhipuai SDK 中封装的 create 函数即可：

In [ ]:
# 请求模型
response = client.chat.completions.create(
    model=model,
    messages=question
)
print(response)

这里对传入 zhipuai 的参数进行简单介绍：

- `messages (list)`: 调用对话模型时，将当前对话信息列表作为提示输入给模型; 按照 {"role": "user", "content": "你好"} 的键值对形式进行传参; 总长度超过模型最长输入限制后会自动截断，需按时间由旧到新排序

- `temperature (float)`: 采样温度，控制输出的随机性，必须为正数取值范围是：(0.0, 1.0)，不能等于 0，默认值为 0.95。值越大，会使输出更随机，更具创造性；值越小，输出会更加稳定或确定
  
- `top_p (float)`: 用温度取样的另一种方法，称为核取样。取值范围是：(0.0, 1.0) 开区间，不能等于 0 或 1，默认值为 0.7。模型考虑具有 top_p 概率质量 tokens 的结果。例如：0.1 意味着模型解码器只考虑从前 10% 的概率的候选集中取 tokens

- `request_id (string)`: 由用户端传参，需保证唯一性；用于区分每次请求的唯一标识，用户端不传时平台会默认生成

- **建议您根据应用场景调整 top_p 或 temperature 参数，但不要同时调整两个参数**
